In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Iniciamos con la lectura de los datos

In [5]:
df = pd.read_excel("/content/real_estate_data.xlsx", index_col = [0])

#### Formateo de los nombres de las columnas para remover espacios

In [6]:
def format_cols_spaces(columns):
  return {key:key.replace(" ","_") for key in columns}

In [11]:
columns_replace = format_cols_spaces(df.columns)
df = df.rename(columns = columns_replace)
df_train = df[0:315]
df_test = df[315:]

### Creamos la clase NeuralNetwork

In [15]:
class NeuralNetwork:
  def __init__(self):

    #Inicializacion de pesos
    self.W1 = np.random.random((5,6))
    self.b1 = np.random.random((5,1))

    self.W2 = np.random.random((1,5))
    self.b2 = np.random.random((1,1))

  def relu(self,X):
    return max(0,X)

  def sigmoid(self):
    return 1/(1+np.exp(-X))

  def forward(self, X):
    zi = self.W1@X + self.b1
    zi = self.sigmoid(zi)
    return zi



In [34]:
neural_1 = NeuralNetwork()

In [17]:
df_train_X = df_train.drop(['Y_house_price_of_unit_area'], axis = 1)
df_train_y = df_train[['Y_house_price_of_unit_area']]

In [31]:
train_X_values = df_train_X['X4_number_of_convenience_stores'].values.reshape(-1,1)


In [35]:
neural_1.W1

array([[0.36683009, 0.6293502 , 0.99593657, 0.1497879 , 0.42186913,
        0.31703935],
       [0.21977489, 0.11666864, 0.22667617, 0.24038088, 0.68081279,
        0.00422961],
       [0.41527001, 0.01281552, 0.43195945, 0.96708445, 0.50210939,
        0.73531882],
       [0.23146256, 0.28133574, 0.60735583, 0.51874061, 0.76207903,
        0.51337657],
       [0.92474077, 0.72435093, 0.25056667, 0.33539024, 0.36360293,
        0.20183351]])

In [ ]:
#Calculo del gradiente numerico

def funcion_objetivo():
    loss =
    return loss

def numerical_gradient():

    gradiente =
    return gradiente

In [ ]:
#funcion fit y loop de entrenamiento
def fit(x, y, learning_rate=0.001, epochs=1000):
    eps = 1e-3
    loss_accum = []
    for i in range(epochs):
        numerical_gradient()

    return loss_accum

In [ ]:
def predict(x):
        y =
        return y